## High-Level ETL (Extract - Transform - Load) Flow
**Goal**: By the end of this tutorial, you will be able to
- Extract: Download a file from AWS S3 using Python’s boto3.
- Transform: Clean, filter, or manipulate data in Python (often using libraries like pandas).
- Load: Insert the transformed data into a relational database via SQL statements.

## Lab Assignment

1. Implement the following functions
   - `extract_from_csv(file_to_process: str) -> pd.DataFrame`: read the .csv file and return dataframe
   - `extract_from_json(file_to_process: str) -> pd.DataFrame`: read the .json file and return dataframe
   - `extract() -> pd.DataFrame`: extract data of heterogeneous format and combine them into a single dataframe.
   - `transform(df) -> pd.DataFrame`: function for data cleaning and manipulation.
2. Clean the data
   - Round float-type columns to two decimal places.
   - remove duplicate samples
   - Save the cleaned data into parquet file
3. Insert the data into SQL
   - Create postgresql database
   - Insert the data into the database
  
Submission requirement:
    1. Jupyter Notebook
    2. Parquet File
    3. SQL file (optional)

In [65]:
# Required Package:
!pip install psycopg2
#pip install pandas 2.0.3 (For data manipulation and analysis)
!pip install sqlalchemy
!pip install pyarrow
import pandas as pd

#required for reading .xml files
import xml.etree.ElementTree as ET

#required for navigating machine's directory
import glob
import os.path

#required for communicating with SQL database
from sqlalchemy import create_engine

  Using cached psycopg2-2.9.10.tar.gz (385 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for psycopg2: filename=psycopg2-2.9.10-cp311-cp311-macosx_11_0_arm64.whl size=132267 sha256=ef5459840ea76a0d9500c347258100ba9eb049d6780728e5ea1c50d573090676
  Stored in directory: /Users/annamcgillis/Library/Caches/pip/wheels/d9/83/60/e9660320860aef3c38a67dea6ff9538e4cad76502cb39ed280
Successfully built psycopg2


# E: Extracting data from multiple sources

This code was removed to allow push to github (secret information)

## Extract data from ./data/ folder

In [30]:
all_files = glob.glob('./data/*')

# Output the list of files
for file in all_files:
    print(file)

./data/used_car_prices1.csv
./data/used_car_prices2.csv
./data/used_car_prices3.csv
./data/used_car_prices1.json
./data/used_car_prices3.xml
./data/used_car_prices2.xml
./data/used_car_prices3.json
./data/used_car_prices1.xml
./data/used_car_prices2.json


### Function to extract data from one .csv file

In [32]:
def extract_from_csv(file_to_process: str) -> pd.DataFrame:
    dataframe = pd.read_csv(file_to_process)
    return dataframe

### Function to extract data from one .json file

In [34]:
def extract_from_json(file_to_process: str) -> pd.DataFrame:
    dataframe = pd.read_json(file_to_process, lines=True)
    return dataframe

### Function to extract data from one  .xml file

In [37]:
def extract_from_xml(file_to_process: str) -> pd.DataFrame:
    dataframe = pd.DataFrame(columns = columns)
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        car_model = person.find("car_model").text
        year_of_manufacture = int(person.find("year_of_manufacture").text)
        price = float(person.find("price").text)
        fuel = person.find("fuel").text
        sample = pd.DataFrame({"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel}, index = [0])
        dataframe = pd.concat([dataframe, sample], ignore_index=True)
    return dataframe

### Function to extract data from the ./data/ folder

In [40]:
def extract() -> pd.DataFrame:
    extracted_data = pd.DataFrame(columns = columns)
    #for csv files
    for csv_file in glob.glob(os.path.join(folder, "*.csv")):
        extracted_data = pd.concat([extracted_data, extract_from_csv(csv_file)], ignore_index=True)
    
    #add lines for json files
    for json_file in glob.glob(os.path.join(folder, "*.json")):
        extracted_data = pd.concat([extracted_data, extract_from_json(json_file)], ignore_index=True)
    
    #add lines for xml files
    for xml_file in glob.glob(os.path.join(folder, "*.xml")):
        extracted_data = pd.concat([extracted_data, extract_from_xml(xml_file)], ignore_index=True)
    
    return extracted_data

### Extract the data

In [47]:
columns = ['car_model','year_of_manufacture','price', 'fuel']
folder = "data"
#table_name = "car_data"

# run
def main():
    data = extract()
    #insert_to_table(data, "car_data")
    
    return data

data = main()

/var/folders/wd/g730h79n6g90zsm1jp44dyv00000gn/T/ipykernel_26257/1959904625.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  extracted_data = pd.concat([extracted_data, extract_from_csv(csv_file)], ignore_index=True)
/var/folders/wd/g730h79n6g90zsm1jp44dyv00000gn/T/ipykernel_26257/3387287639.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe = pd.concat([dataframe, sample], ignore_index=True)
/var/folders/wd/g730h79n6g90zsm1jp44dyv00000gn/T/ipykernel_26257/3387287639.py:11: Future

In [49]:
data.head()

car_model year_of_manufacture         price    fuel
0      ritz                2014   5000.000000  Petrol
1       sx4                2013   7089.552239  Diesel
2      ciaz                2017  10820.895522  Petrol
3   wagon r                2011   4253.731343  Petrol
4     swift                2014   6865.671642  Diesel

# T: Transformation data and save organized data to .parquet file 

In [52]:
staging_file = "cars.parquet"
staging_data_dir = "staging_data"

In [54]:
def transform(df):
    print(f"Shape of data {df.shape}")

    # truncate price with 2 decimal place (add your code below)
    df['price'] = df['price'].apply(lambda x: round(x, 2))

    # remove samples with same car_model (add your code below)
    df = df.drop_duplicates(subset=['car_model'])
    
    print(f"Shape of data {df.shape}")

    # Ensure the staging directory exists before writing the Parquet file
    if not os.path.exists(staging_data_dir):
        os.makedirs(staging_data_dir)
        print(f"Directory '{staging_data_dir}' created.")

    # write to parquet
    df.to_parquet(os.path.join(staging_data_dir, staging_file))
    return df

In [56]:
# print the head of your data
df = transform(data)
df.head()

Shape of data (90, 4)
Shape of data (25, 4)
Directory 'staging_data' created.


car_model year_of_manufacture     price    fuel
0      ritz                2014   5000.00  Petrol
1       sx4                2013   7089.55  Diesel
2      ciaz                2017  10820.90  Petrol
3   wagon r                2011   4253.73  Petrol
4     swift                2014   6865.67  Diesel

# L: Loading data for further modeling

### Set Up PostgreSQL Locally
#### Step 1: Install PostgreSQL
- Windows: Download from MySQL Official Site {https://www.postgresql.org/download/}
- Mac:
  ```{bash}
  brew install postgresql
  brew services start postgresql
  ```
Then access PostgreSQL CLI
```{bash}
psql -U postgres
```
Note: if you don't have default "postgres" user, then create it manually by
```{bash}
default "postgres" user
```
or
```{bash}
sudo -u $(whoami) createuser postgres -s
```

Then create a database
```{sql}
CREATE DATABASE my_local_db;
\l  -- List all databases
```

#### Step 2: Create a User and Grant Privileges
In PostgreSQL CLI:
```{sql}
CREATE USER myuser WITH ENCRYPTED PASSWORD 'mypassword';
GRANT ALL PRIVILEGES ON DATABASE my_local_db TO myuser;
```

#### Step 3: Install Required Python Libraries
```{bash}
pip install pandas sqlalchemy pymysql psycopg2 mysql-connector-python
```

### Utility function for writing data into the SQL database

In [67]:
# Database credentials
db_host = "localhost"
db_user = "myuser"
db_password = "mypassword"
db_name = "my_local_db"

conn_string = f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}/{db_name}"

engine = create_engine(conn_string)

In [69]:
# Test connection
df = pd.read_sql("SELECT * FROM pg_catalog.pg_tables;", con=engine)
print(df)

            schemaname                tablename    tableowner tablespace  \
0           pg_catalog             pg_statistic  annamcgillis       None   
1           pg_catalog                  pg_type  annamcgillis       None   
2           pg_catalog         pg_foreign_table  annamcgillis       None   
3           pg_catalog                pg_authid  annamcgillis  pg_global   
4           pg_catalog    pg_statistic_ext_data  annamcgillis       None   
..                 ...                      ...           ...        ...   
61          pg_catalog           pg_largeobject  annamcgillis       None   
62  information_schema                sql_parts  annamcgillis       None   
63  information_schema  sql_implementation_info  annamcgillis       None   
64  information_schema             sql_features  annamcgillis       None   
65  information_schema               sql_sizing  annamcgillis       None   

    hasindexes  hasrules  hastriggers  rowsecurity  
0         True     False        Fa

In [71]:
def insert_to_table(data: pd.DataFrame, conn_string:str, table_name:str):
    db = create_engine(conn_string) # creates a connection to the database using SQLAlchemy
    conn = db.connect() # Establishes a database connection
    data.to_sql(table_name, conn, if_exists="replace", index=False)
    conn.close()

In [73]:
# read from the .parquet file

def load() -> pd.DataFrame:
    data = pd.DataFrame()
    for parquet_file in glob.glob(os.path.join(staging_data_dir, "*.parquet")):
        data = pd.concat([pd.read_parquet(parquet_file),data])

    #insert_to_table(data, table_name)
    insert_to_table(data = data, conn_string = conn_string, table_name = 'ml_car_data')

    return data

data = load()
print(data.shape)

(25, 4)
